In [1]:
import mesa
import numpy as np
from enum import Enum
import uuid
import random

def new_uuid():
    # Generate a UUID
    unique_id = uuid.uuid4()

    # Convert the UUID to an integer
    int_uuid = int(unique_id.int)

    return int_uuid

For SEIR model rigth now we assume, that mosquito can be in SUSCEPTIBLE, EXPOSED or INFECTED state. If mosquito is INFECTED it stays like that for the rest of it's life.

Human after beeing INFECTED for `infection_period` has some probability of recovering. Right now we assume that reovered human is immune to malaria and can be exposed any more.

In [2]:
import mesa
import uuid
from enum import Enum
from mesa.model import Model
import random
from copy import copy


def new_uuid():
    # Generate a UUID
    unique_id = uuid.uuid4()

    # Convert the UUID to an integer
    int_uuid = int(unique_id.int)

    return int_uuid


class SEIR(Enum):
    SUSCEPTIBLE = 1
    EXPOSED = 2
    INFECTED = 3
    RECOVERED = 4


class LIFE_STAGE(Enum):
    LARVAE = 1
    ADULT = 2


class HumanAgent(mesa.Agent):
    """
    If a mosquito bites a human, then the MosquitoAgent is responsible for changing human's seir to EXPOSED. After incubation period
    human's state is changed to INFECTED. After infection_period human has recovery_probability of changing to RECOVERED. If human
    is not recovered then it dies
    """

    def __init__(self, unique_id, model, seir: SEIR = SEIR.SUSCEPTIBLE, **kwargs):
        super().__init__(unique_id, model)
        self.incubation_period = random.randint(kwargs["human_incubation_period_range"][0],
                                                kwargs["human_incubation_period_range"][1])
        self.recovery_probability = kwargs["human_recovery_probability_multiplier"]
        self.susceptible_probability = kwargs["human_susceptible_probability_multiplier"]

        self.time_exposed = 0
        self.time_infected = 0
        self.time_recovered = 0
        self.prev_day = copy(model.day_count)
        self.day_of_infection = None if seir == SEIR.SUSCEPTIBLE or seir == SEIR.RECOVERED else copy(model.day_count)
        self.day_of_recovery = None if seir != SEIR.RECOVERED else copy(model.day_count)
        self.seir = seir
        self.type = "Human"

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False)
        new_position = self.random.choice(list(possible_steps))
        self.model.grid.move_agent(self, new_position)

    def check_seir(self):
        dead = False
        if self.seir == SEIR.EXPOSED:
            if self.time_exposed < self.incubation_period:
                if self.prev_day != self.model.day_count:
                    self.time_exposed += 1
            else:
                self.seir = SEIR.INFECTED
                self.time_exposed = 0
                self.day_of_infection = copy(self.model.day_count)
                self.time_infected = 0
        elif self.seir == SEIR.INFECTED:
            # if self.time_infected < self.infection_period:
            #     if self.prev_day != self.model.day_count:
            #         self.time_infected += 1
            # else:
            #     if random.random() < self.recovery_probability * (self.model.day_count - self.day_of_infection):
            #         self.time_infected = 0
            #         self.seir = SEIR.RECOVERED
            #         self.day_of_recovery = copy(self.model.day_count)
            #     else:
            #         self.die()
            #         dead = True
            if self.prev_day != self.model.day_count:
                self.time_infected += 1
                if random.random() < self.recovery_probability * self.time_infected:
                    self.time_infected = 0
                    self.time_recovered = 0
                    self.seir = SEIR.RECOVERED
                    self.day_of_recovery = copy(self.model.day_count)
        elif self.seir == SEIR.RECOVERED:
            if random.random() < self.susceptible_probability * self.time_recovered:
                self.seir = SEIR.SUSCEPTIBLE
                self.time_recovered = 0
            else:
                if self.prev_day != self.model.day_count:
                    self.time_recovered += 1
        return dead

    def die(self):
        self.model.schedule.remove(self)
        self.model.grid.remove_agent(self)

    def step(self):
        if self.check_seir():
            return
        self.move()
        self.prev_day = copy(self.model.day_count)


class MosquitoAgent(mesa.Agent):
    """
    Mosquitos can be in one of two life stages LARVAE or ADULT. Only adult mosquitos can move and be malaria vectors.

    If self.looking_for_water is equal to False, then mosquito is looking for a human. When mosquito is looking for a human it moves one cell
    in random direction until it lands in one cell with a human. After biting a human, mosquito changes self.looking_for_water to True.
    Now it is moving in random direction to find a water source and it doesn't care about humans.

    If it finds a cell with other agent of type "Water" it creates some number of new LARVAE mosquitos in this cell. After
    creating new larves, mosquito changes self.looking_for_water again to False and again searches for human. 

    If mosquito is in one cell with an agent of type "House" and the house has net or spray, it has some chance of being repelled 
    (if mosquito is repelled it doesn't die and doesn't bite, just continues to move). If it wasn't repelled then there is a chance of
    killing a mosquito. If mosquito both isn't repelled and doesn't die, then it can still bite a human if the human is also present in 
    this cell

    """

    def __init__(self, unique_id, model, life_stage: LIFE_STAGE = LIFE_STAGE.ADULT, seir: SEIR = SEIR.SUSCEPTIBLE,
                 **kwargs):
        super().__init__(unique_id, model)
        self.kwargs_from_init = kwargs # safe kwargs in order to pass them while creating eggs
        # Life stages
        self.life_stage = life_stage
        self.larvae_period = random.randint(kwargs["mosquito_larvae_period_range"][0],
                                            kwargs["mosquito_larvae_period_range"][1])
        if life_stage == LIFE_STAGE.ADULT:
            self.current_life_step = self.larvae_period
        else:
            self.current_life_step = 0
        self.life_time = self.larvae_period + random.randint(kwargs["mosquito_adult_life_range"][0],
                                                             kwargs["mosquito_adult_life_range"][1])

        # Reproduction
        self.daily_min_eggs_laid = kwargs["mosquito_daily_min_eggs_laid"]
        self.daily_max_eggs_laid = kwargs["mosquito_daily_max_eggs_laid"]
        self.lifetime_max_eggs = kwargs["mosquito_lifetime_max_eggs"]
        self.eggs_laid_during_day = 0
        self.total_eggs_laid = 0

        # Incubation and infections
        self.time_exposed = 0
        self.seir = seir
        self.incubation_period = random.randint(kwargs["mosquito_incubation_period_range"][0],
                                                kwargs["mosquito_incubation_period_range"][1])
        self.probability_of_exposition = kwargs["mosquito_probability_of_exposition"]
        self.probability_of_infecting_human = kwargs["mosquito_probability_of_infecting_human"]

        # Movement
        self.daily_steps_available = kwargs["mosquito_daily_steps"]
        self.remaining_steps = kwargs["mosquito_daily_steps"]

        self.type = "Mosquito"
        self.looking_for_water = False
        self.prev_day = copy(model.day_count)

    def move(self):
        if self.life_stage == LIFE_STAGE.ADULT and self.remaining_steps > 0:
            possible_steps = self.model.grid.get_neighborhood(
                self.pos, moore=True, include_center=False)
            new_position = self.random.choice(possible_steps)
            self.remaining_steps -= 1
            self.model.grid.move_agent(self, new_position)

    def reset_steps(self):
        self.remaining_steps = self.daily_steps_available

    def reset_eggs(self):
        self.eggs_laid_during_day = 0

    def bite(self, human: HumanAgent):
        txt = f"before bite m={self.seir}, h={human.seir}"
        if human.seir == SEIR.INFECTED and self.seir == SEIR.SUSCEPTIBLE:
            if random.random() < self.probability_of_exposition:
                self.seir = SEIR.EXPOSED
        elif self.seir == SEIR.INFECTED and human.seir == SEIR.SUSCEPTIBLE:
            if random.random() < self.probability_of_infecting_human:
                human.seir = SEIR.EXPOSED
        txt += f" | after bite m={self.seir}, h={human.seir}"

    def die(self):
        self.model.schedule.remove(self)
        self.model.grid.remove_agent(self)

    def check_life_stage(self):
        dead_or_larvae = False
        if self.life_stage == LIFE_STAGE.LARVAE and self.current_life_step < self.larvae_period:
            dead_or_larvae = True
        elif self.life_stage == LIFE_STAGE.LARVAE and self.current_life_step >= self.larvae_period:
            # change from larvae to adult
            self.life_stage = LIFE_STAGE.ADULT
        elif self.life_stage == LIFE_STAGE.ADULT and self.current_life_step >= self.life_time:
            self.die()
            dead_or_larvae = True
        return dead_or_larvae

    def check_seir(self):
        if self.seir == SEIR.EXPOSED:
            if self.time_exposed < self.incubation_period:
                if self.prev_day != self.model.day_count:
                    self.time_exposed += 1
            else:
                self.seir = SEIR.INFECTED
                self.time_exposed = 0

    def lay_eggs(self):
        number_of_eggs = random.randint(self.daily_min_eggs_laid, self.daily_max_eggs_laid)
        for _ in range(number_of_eggs):
            if self.eggs_laid_during_day >= self.daily_max_eggs_laid:
                break
            a = MosquitoAgent(new_uuid(), self.model, life_stage=LIFE_STAGE.LARVAE,
                              seir=self.seir, **self.kwargs_from_init)
            self.eggs_laid_during_day += 1
            self.total_eggs_laid += 1
            self.model.new_mosquitos.append((a, self.pos))

    def bite_or_eggs(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if self.looking_for_water and self.total_eggs_laid < self.lifetime_max_eggs:
            for c in cellmates:
                if c.type == "Water":
                    self.lay_eggs()
                    self.looking_for_water = False
                    break
        else:
            for c in cellmates:
                if c.type == "Human":
                    self.bite(c)
                    if self.total_eggs_laid < self.lifetime_max_eggs:
                        self.looking_for_water = True
                    break

    def check_house_net(self):
        dead_or_repelled = False
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        house = None
        for c in cellmates:
            if c.type == "House":
                house = c
                break
        if house is not None:
            action = "enter"
            if house.mosquito_spray:
                action = random.choice(["repel", "kill", "enter"])
            if action == "kll":
                dead_or_repelled = True
                self.die()
            elif action == "repel":
                dead_or_repelled = True
                self.move()
        return dead_or_repelled

    def check_house_spray(self):
        dead_or_repelled = False
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        house = None
        for c in cellmates:
            if c.type == "House":
                house = c
                break
        if house is not None:
            action = "enter"
            if house.mosquito_net:
                action = random.choice(["repel", "kill", "enter"])
            if action == "kill":
                dead_or_repelled = True
                self.die()
            elif action == "repel":
                dead_or_repelled = True
                self.move()
        return dead_or_repelled

    def step(self):
        if self.prev_day != self.model.day_count:
            self.current_life_step += 1
            self.reset_steps()
            self.reset_eggs()
        self.prev_day = copy(self.model.day_count)
        if self.check_life_stage():
            return
        self.check_seir()
        self.move()
        if self.check_house_net():
            return
        self.bite_or_eggs()
        self.check_house_spray()


class HouseAgent(mesa.Agent):
    """Agent representing a house. It doesn't move"""

    def __init__(self, unique_id, model, mosquito_net: bool, mosquito_spray: bool):
        super().__init__(unique_id, model)
        self.mosquito_net = mosquito_net
        self.mosquito_spray = mosquito_spray
        self.type = "House"

    def step(self):
        pass


class WaterAgent(mesa.Agent):
    """Agent representing a water source. It doesn't move and doesn't have any extra properties"""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = "Water"

    def step(self):
        pass



In [3]:
class MalariaInfectionModel(mesa.Model):
    """A model with some number of agents."""
    def __init__(self, **kwargs):

        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.MultiGrid(kwargs["width"], kwargs["height"], True)
        self.day_count = 0  # number of day
        self.day_step = 0  # each day has 24 simulation steps
        self.initial_humans = kwargs["initial_humans"]
        self.new_mosquitos = []  # list for storing new mosquitos to add

        self.datacollector = mesa.DataCollector(
            {
                "Humans": lambda m: sum([1 for agent in m.schedule.agents if isinstance(agent, HumanAgent)]),
                "Mosquitos": lambda m: sum([1 for agent in m.schedule.agents if isinstance(agent, MosquitoAgent)]),
            }
        )

        # Create human agents
        infected_humans = int(kwargs["percentage_of_infected_humans"] * kwargs["initial_humans"])
        susceptible_humans = kwargs["initial_humans"] - infected_humans
        for _ in range(infected_humans):
            a = HumanAgent(new_uuid(), self, seir=SEIR.INFECTED, **kwargs)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)
        for _ in range(susceptible_humans):
            a = HumanAgent(new_uuid(), self, seir=SEIR.SUSCEPTIBLE, **kwargs)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

        # Create mosquito agents
        infected_mosquitos = int(kwargs["percentage_of_infected_mosquitos"] * kwargs["initial_mosquitos"])
        susceptible_mosquitos = kwargs["initial_mosquitos"] - infected_mosquitos
        for _ in range(infected_mosquitos):
            a = MosquitoAgent(new_uuid(), self, life_stage=random.choice(list(LIFE_STAGE)),
                              seir=SEIR.INFECTED, **kwargs)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)
        for _ in range(susceptible_mosquitos):
            a = MosquitoAgent(new_uuid(), self, life_stage=random.choice(list(LIFE_STAGE)),
                              seir=SEIR.SUSCEPTIBLE, **kwargs)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

        # Create house agents
        for _ in range(kwargs["houses"]):
            a = HouseAgent(new_uuid(), self, random.choice([True, False]), random.choice([True, False]))

            collision_with_house_or_water = True

            while collision_with_house_or_water:
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                cellmates = self.grid.get_cell_list_contents([(x, y)])
                if all(c.type != "House" and c.type != "Water" for c in cellmates):
                    collision_with_house_or_water = False

            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

        # Create water agents
        for _ in range(kwargs["ponds"]):
            a = WaterAgent(new_uuid(), self)

            collision_with_house_or_water = True

            while collision_with_house_or_water:
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                cellmates = self.grid.get_cell_list_contents([(x, y)])
                if all(c.type != "House" and c.type != "Water" for c in cellmates):
                    collision_with_house_or_water = False

            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        self.day_step += 1
        for m in self.new_mosquitos:
            self.grid.place_agent(m[0], m[1])
            self.schedule.add(m[0])
        self.new_mosquitos = []  # clear the list for the next step
        if self.day_step == 24:
            self.day_step = 0
            self.day_count += 1

    def count_infected_humans(self):
        return sum(1 for agent in self.schedule.agents if isinstance(agent, HumanAgent) and agent.seir == SEIR.INFECTED)

    def count_susceptible_humans(self):
        return sum(
            1 for agent in self.schedule.agents if isinstance(agent, HumanAgent) and agent.seir == SEIR.SUSCEPTIBLE)

    def count_exposed_humans(self):
        return sum(1 for agent in self.schedule.agents if isinstance(agent, HumanAgent) and agent.seir == SEIR.EXPOSED)

    def count_recovered_humans(self):
        return sum(
            1 for agent in self.schedule.agents if isinstance(agent, HumanAgent) and agent.seir == SEIR.RECOVERED)

    def count_infected_mosquitos(self):
        return sum(
            1 for agent in self.schedule.agents if isinstance(agent, MosquitoAgent) and agent.seir == SEIR.INFECTED)

    def count_susceptible_mosquitos(self):
        return sum(
            1 for agent in self.schedule.agents if isinstance(agent, MosquitoAgent) and agent.seir == SEIR.SUSCEPTIBLE)

    def count_exposed_mosquitos(self):
        return sum(
            1 for agent in self.schedule.agents if isinstance(agent, MosquitoAgent) and agent.seir == SEIR.EXPOSED)

    def count_adult_mosquitos(self):
        return sum(1 for agent in self.schedule.agents if
                   isinstance(agent, MosquitoAgent) and agent.life_stage == LIFE_STAGE.ADULT)

    def count_humans(self):
        human_counter = 0
        for agent in self.schedule.agents:
            if isinstance(agent, HumanAgent):
                human_counter += 1
        return human_counter

    def count_deaths(self):
        actual_humans = self.count_humans()
        deaths = self.initial_humans - actual_humans
        return deaths




In [4]:
default_parameters = {
    "width": 1000,
    "height": 1000,
    "initial_mosquitos":10000, 
    "initial_humans": 10000,
    "houses": 1000,
    "ponds": 10,
    "percentage_of_infected_humans": 0.4,
    "percentage_of_infected_mosquitos": 0.2,
    
    "human_incubation_period_range": [7, 30],
    "human_recovery_probability_multiplier": 0.037,
    "human_susceptible_probability_multiplier": 0.01,

    "mosquito_larvae_period_range": [9, 14],
    "mosquito_adult_life_range": [7, 30],
    "mosquito_daily_min_eggs_laid": 50,
    "mosquito_daily_max_eggs_laid": 200,
    "mosquito_lifetime_max_eggs": 500,
    "mosquito_incubation_period_range": [10, 21],
    "mosquito_probability_of_exposition": 0.02,
    "mosquito_probability_of_infecting_human": 0.5,
    "mosquito_daily_steps": 10,
}

In [ ]:
parameters = {
    "width": 100,
    "height": 100,
    "initial_mosquitos":7000, 
    "initial_humans": 1000,
    "houses": 20,
    "ponds": 10,
    "percentage_of_infected_humans": 0.4,
    "percentage_of_infected_mosquitos": 0.2,
    
    "human_incubation_period_range": [7, 30],
    "human_recovery_probability_multiplier": 0.037,
    "human_susceptible_probability_multiplier": 0.01,

    "mosquito_larvae_period_range": [9, 14],
    "mosquito_adult_life_range": [7, 30],
    "mosquito_daily_min_eggs_laid": 50,
    "mosquito_daily_max_eggs_laid": 200,
    "mosquito_lifetime_max_eggs": 500,
    "mosquito_incubation_period_range": [10, 21],
    "mosquito_probability_of_exposition": 0.02,
    "mosquito_probability_of_infecting_human": 0.5,
    "mosquito_daily_steps": 10,
}


starter_model = MalariaInfectionModel(**parameters)


initial_infected_count = starter_model.count_infected_humans()
print("Initial Number of Infected Humans:", initial_infected_count)
initial_susceptible_count = starter_model.count_susceptible_humans()
print("Initial Number of Susceptible Humans:", initial_susceptible_count)

# Symulacja przez 60 dni
day = -1
for i in range(2*720):
    if day != starter_model.day_count:
        day = starter_model.day_count
        print(f"day {day}")
        print(f"total mosquitos: {starter_model.count_infected_mosquitos()+starter_model.count_exposed_mosquitos()+starter_model.count_susceptible_mosquitos()}")
        print(f"infected mosquitos: {starter_model.count_infected_mosquitos()}")
        print(f"exposed mosquitos: {starter_model.count_exposed_mosquitos()}")
        print(f"adul mosquitos: {starter_model.count_adult_mosquitos()}")
        print(f"infected humans: {starter_model.count_infected_humans()}")
        print(f"exposed humans: {starter_model.count_exposed_humans()}")
        print("\n")
    starter_model.step()

final_infected_count = starter_model.count_infected_humans()
print("Final Number of Infected Humans:", final_infected_count)
final_susceptible_count = starter_model.count_susceptible_humans()
print("Final Number of Susceptible Humans:", final_susceptible_count)
final_exposed_count = starter_model.count_exposed_humans()
print("Final Number of Exposed Humans:", final_exposed_count)
final_recovered_count = starter_model.count_infected_humans()
print("Final Number of Recovered Humans:", final_recovered_count)
final_deaths_count = starter_model.count_deaths()
print("Deaths: ", final_deaths_count)



Initial Number of Infected Humans: 400
Initial Number of Susceptible Humans: 600
day 0
total mosquitos: 7000
infected mosquitos: 0
exposed mosquitos: 0
adul mosquitos: 3426
infected humans: 400
exposed humans: 0


day 1
total mosquitos: 8167
infected mosquitos: 0
exposed mosquitos: 26
adul mosquitos: 3415
infected humans: 400
exposed humans: 0


day 2
total mosquitos: 11310
infected mosquitos: 0
exposed mosquitos: 35
adul mosquitos: 3402
infected humans: 388
exposed humans: 0


day 3
total mosquitos: 14091
infected mosquitos: 0
exposed mosquitos: 37
adul mosquitos: 3387
infected humans: 356
exposed humans: 0


day 4
total mosquitos: 16903
infected mosquitos: 0
exposed mosquitos: 37
adul mosquitos: 3379
infected humans: 314
exposed humans: 0


day 5
total mosquitos: 20430
infected mosquitos: 0
exposed mosquitos: 148
adul mosquitos: 3362
infected humans: 261
exposed humans: 0


day 6
total mosquitos: 24795
infected mosquitos: 0
exposed mosquitos: 148
adul mosquitos: 3356
infected humans: